In [45]:
import random
import torch
import numpy as np

In [46]:
from microservice.models.model_b import SimpleNN
from models.train_utils_nn import train_and_extract
from models.utils import load_default_data, calculate_expenses
from microservice.features import extract_users_data, prepare_data_for_predict
from sklearn.metrics import roc_auc_score as roc
from microservice.models.model_b import NeuralNetworkRegressor
from models.train_utils_nn import train


In [47]:
seed = 213769420
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [48]:
train_sessions, test, products, users, deliveries = load_default_data()
targets = calculate_expenses(test, products, users)
targets

,user_id,expenses
0,102,0.00
1,103,554.77
2,104,2332.01
3,105,0.00
4,106,0.00
...,...,...
195,297,109.00
196,298,2399.00
197,299,0.00
198,300,0.00


In [49]:
targets['expenses'] = targets['expenses'].apply(lambda b: 1 if b > 5000 else 0)

In [50]:
targets

,user_id,expenses
0,102,0
1,103,0
2,104,0
3,105,0
4,106,0
...,...,...
195,297,0
196,298,0
197,299,0
198,300,0


In [51]:
some_net = NeuralNetworkRegressor()
users_data = extract_users_data(train_sessions, users, products)
users_data

,user_id,expenses,products_bought,events_number,sessions_number,average_discount,average_discount_on_bought,city
0,139,23400.85,49,255,66,11.019608,10.816327,Kraków
1,242,44518.18,58,327,86,11.223242,10.603448,Radom
2,108,16312.03,23,129,32,9.418605,9.347826,Radom
3,143,7214.08,15,84,26,11.250000,11.666667,Kraków
4,140,32333.55,43,306,83,10.130719,9.651163,Poznań
...,...,...,...,...,...,...,...,...
195,289,0.00,0,7,1,20.000000,0.000000,Warszawa
196,225,109.00,1,2,1,10.000000,10.000000,Warszawa
197,152,0.00,0,5,2,2.000000,0.000000,Poznań
198,162,78.96,2,6,2,13.333333,12.500000,Szczecin


In [52]:
train(some_net, users_data, targets)

Epoch 0 loss 0.0848
Epoch 1 loss 0.0924
Epoch 2 loss 0.0658
Epoch 3 loss 0.0976
Epoch 4 loss 0.0729
Epoch 5 loss 0.0561
Epoch 6 loss 0.0883
Epoch 7 loss 0.0687
Epoch 8 loss 0.0851
Epoch 9 loss 0.0893
Epoch 10 loss 0.0939
Epoch 11 loss 0.0646
Epoch 12 loss 0.0803
Epoch 13 loss 0.0653
Epoch 14 loss 0.0702
Epoch 15 loss 0.0523
Epoch 16 loss 0.0984
Epoch 17 loss 0.0715
Epoch 18 loss 0.0658
Epoch 19 loss 0.0577
Epoch 20 loss 0.0659
Epoch 21 loss 0.0614
Epoch 22 loss 0.0814
Epoch 23 loss 0.0606
Epoch 24 loss 0.059
Epoch 25 loss 0.0811
Epoch 26 loss 0.0731
Epoch 27 loss 0.0794
Epoch 28 loss 0.1
Epoch 29 loss 0.0998
Epoch 30 loss 0.0652
Epoch 31 loss 0.052
Epoch 32 loss 0.0617
Epoch 33 loss 0.0578
Epoch 34 loss 0.0456
Epoch 35 loss 0.0403
Epoch 36 loss 0.069
Epoch 37 loss 0.0519
Epoch 38 loss 0.0698
Epoch 39 loss 0.0528
Epoch 40 loss 0.0757
Epoch 41 loss 0.0606
Epoch 42 loss 0.0485
Epoch 43 loss 0.0651
Epoch 44 loss 0.0753
Epoch 45 loss 0.0476
Epoch 46 loss 0.069
Epoch 47 loss 0.0535
Epoch 48 

In [53]:
extracted_users_data = extract_users_data(train_sessions, users, products)
x, cat_x = prepare_data_for_predict(extracted_users_data)
x = torch.from_numpy(x.values).float()
cat_x = torch.from_numpy(cat_x.values).float()
some_net.eval()
out = some_net(x, cat_x).squeeze()
out = out.detach().numpy()
out

array([0.09270999, 0.0727239 , 0.10107204, 0.08470789, 0.08265649,
       0.09702913, 0.08275331, 0.09776396, 0.08432503, 0.06179321,
       0.08911065, 0.0877873 , 0.09565176, 0.04253358, 0.08802857,
       0.10045748, 0.08061907, 0.09346148, 0.0873265 , 0.04867771,
       0.05444264, 0.09277996, 0.0856462 , 0.06962235, 0.08719957,
       0.08329393, 0.08676489, 0.06964958, 0.04963084, 0.08980252,
       0.10021158, 0.09165062, 0.08895054, 0.10011285, 0.07719723,
       0.06765188, 0.08907329, 0.07970862, 0.07408775, 0.09056626,
       0.09018449, 0.08842079, 0.08821931, 0.09289826, 0.04800501,
       0.09699097, 0.05973797, 0.09149396, 0.07129056, 0.08980827,
       0.08759783, 0.05257551, 0.08947656, 0.0939365 , 0.04317959,
       0.04396026, 0.09877651, 0.08319824, 0.08798052, 0.09834658,
       0.09059656, 0.08477752, 0.09890369, 0.09644812, 0.07484641,
       0.10007598, 0.07441878, 0.04285541, 0.09961978, 0.09750169,
       0.09550383, 0.10031796, 0.08153098, 0.09116466, 0.09281

In [54]:
out_array = {int(user_id): out[i] > 0.3 for i, user_id in enumerate(extracted_users_data["user_id"].to_list())}
out_array

{139: False,
 242: False,
 108: False,
 143: False,
 140: False,
 202: False,
 270: False,
 212: False,
 189: False,
 205: False,
 255: False,
 215: False,
 273: False,
 281: False,
 227: False,
 222: False,
 235: False,
 171: False,
 188: False,
 135: False,
 252: False,
 238: False,
 145: False,
 286: False,
 157: False,
 265: False,
 127: False,
 159: False,
 119: False,
 274: False,
 160: False,
 105: False,
 261: False,
 185: False,
 183: False,
 192: False,
 186: False,
 102: False,
 124: False,
 282: False,
 210: False,
 125: False,
 272: False,
 104: False,
 115: False,
 136: False,
 285: False,
 133: False,
 187: False,
 175: False,
 161: False,
 167: False,
 213: False,
 292: False,
 256: False,
 278: False,
 263: False,
 148: False,
 200: False,
 280: False,
 287: False,
 208: False,
 149: False,
 233: False,
 236: False,
 246: False,
 193: False,
 147: False,
 158: False,
 251: False,
 277: False,
 182: False,
 151: False,
 253: False,
 295: False,
 262: False,
 178: False,

In [55]:
targets.values

array([[102,   0],
       [103,   0],
       [104,   0],
       [105,   0],
       [106,   0],
       [107,   0],
       [108,   0],
       [109,   0],
       [110,   0],
       [111,   0],
       [112,   0],
       [113,   0],
       [114,   0],
       [115,   0],
       [116,   0],
       [117,   0],
       [118,   0],
       [119,   0],
       [120,   0],
       [121,   0],
       [122,   0],
       [123,   0],
       [124,   0],
       [125,   0],
       [126,   0],
       [127,   0],
       [128,   0],
       [129,   0],
       [130,   0],
       [131,   0],
       [132,   1],
       [133,   0],
       [134,   0],
       [135,   0],
       [136,   0],
       [137,   0],
       [138,   0],
       [139,   0],
       [140,   0],
       [141,   0],
       [142,   0],
       [143,   0],
       [144,   0],
       [145,   0],
       [146,   0],
       [147,   0],
       [148,   0],
       [149,   0],
       [150,   0],
       [151,   0],
       [152,   0],
       [153,   0],
       [154,

In [56]:
numerator = 0
for row in targets.values[:]:
    if row[1] == out_array[row[0]]:
        print(row[0])
        numerator += 1

102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
165
166
167
168
169
170
171
172
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
267
268
269
270
271
272
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301


In [57]:
some_net(x, cat_x).detach()

tensor([[0.0927],
        [0.0727],
        [0.1011],
        [0.0847],
        [0.0827],
        [0.0970],
        [0.0828],
        [0.0978],
        [0.0843],
        [0.0618],
        [0.0891],
        [0.0878],
        [0.0957],
        [0.0425],
        [0.0880],
        [0.1005],
        [0.0806],
        [0.0935],
        [0.0873],
        [0.0487],
        [0.0544],
        [0.0928],
        [0.0856],
        [0.0696],
        [0.0872],
        [0.0833],
        [0.0868],
        [0.0696],
        [0.0496],
        [0.0898],
        [0.1002],
        [0.0917],
        [0.0890],
        [0.1001],
        [0.0772],
        [0.0677],
        [0.0891],
        [0.0797],
        [0.0741],
        [0.0906],
        [0.0902],
        [0.0884],
        [0.0882],
        [0.0929],
        [0.0480],
        [0.0970],
        [0.0597],
        [0.0915],
        [0.0713],
        [0.0898],
        [0.0876],
        [0.0526],
        [0.0895],
        [0.0939],
        [0.0432],
        [0

In [58]:
roc(targets.expenses, some_net(x, cat_x).detach())

0.6915807560137457

In [59]:
torch.save(some_net.state_dict(), "../models/dummy_test")